# 🌦️ WeatherWise






## 🧰 Setup and Imports



In [56]:

!pip install pyinputplus



Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unloc

## 📦 Setup and Configuration
Import required packages and setup environment.

In [57]:
import requests
from requests.utils import quote
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta
import calendar
import pyinputplus as pyip



## 🌤️ Weather Data Functions

In [58]:


def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location from wttr.in API.
    Handles edge cases like invalid input and special characters.
    """
    # Validate inputs
    if not location.strip():
        raise ValueError("Location cannot be empty")
    if forecast_days < 1 or forecast_days > 5:
        raise ValueError("forecast_days must be between 1 and 5")

    try:
        url = f"https://wttr.in/{quote(location)}?format=j1"
        response = requests.get(url)
        response.raise_for_status()
        raw_data = response.json()

        # Check for valid response
        if "current_condition" not in raw_data or not raw_data["current_condition"]:
            print(f"No weather data available for '{location}'.")
            return {}

        current = raw_data["current_condition"][0]
        forecast = raw_data.get("weather", [])[:forecast_days]

        weather_data = {
            "location": location,
            "current": {
                "temperature_C": current.get("temp_C"),
                "condition": current.get("weatherDesc", [{}])[0].get("value"),
                "humidity": current.get("humidity"),
                "feels_like_C": current.get("FeelsLikeC"),
            },
            "forecast": []
        }

        for day in forecast:
            day_info = {
                "date": day.get("date"),
                "max_temp_C": day.get("maxtempC"),
                "min_temp_C": day.get("mintempC"),
                "hourly": []
            }

            for hour in day.get("hourly", []):
                day_info["hourly"].append({
                    "time": hour.get("time"),
                    "temp_C": hour.get("tempC"),
                    "precip_mm": hour.get("precipMM"),
                    "chance_of_rain": hour.get("chanceofrain"),
                    "condition": hour.get("weatherDesc", [{}])[0].get("value"),
                })

            weather_data["forecast"].append(day_info)

        return weather_data

    except requests.RequestException as e:
        print(f"Error fetching data from wttr.in: {e}")
        return {}



## 🤖 Natural Language Processing

In [59]:
# --- helpers / constants ---
DATE_WORDS = {
    "today", "tomorrow", "tonight", "now",
    "this", "next", "weekend",
    # weekday names will be added dynamically below
}
WEEKDAYS = [d.lower() for d in calendar.day_name]  # monday, tuesday, ...
DATE_WORDS.update(WEEKDAYS)

# used in the regex positive lookahead to stop capturing location before any of these words
_STOP_WORDS_PATTERN = r"(?:today|tomorrow|tonight|now|this|next|on|for|at|in|during|am|pm|morning|afternoon|evening|" + \
                      "|".join(WEEKDAYS) + r")"


def _parse_date_from_text(text):
    """Return a datetime.date object for 'today', 'tomorrow', or next weekday if mentioned. Defaults to today."""
    text_l = text.lower()
    today = datetime.today().date()

    if re.search(r"\btoday\b", text_l) or re.search(r"\bnow\b", text_l) or re.search(r"\btonight\b", text_l):
        return today

    if re.search(r"\btomorrow\b", text_l):
        return today + timedelta(days=1)

    # check for weekday names
    for i, wd in enumerate(calendar.day_name):  # Monday..Sunday
        if re.search(r"\b" + wd.lower() + r"\b", text_l):
            # compute next date for that weekday (could be today if weekday matches)
            days_ahead = (i - today.weekday()) % 7
            # if they asked "Monday" and today is Monday, assume they mean today; you could choose +7 to mean next week
            return today + timedelta(days=days_ahead)

    # default
    return today


def _extract_location(user_question: str) -> str | None:
    """
    Extract a clean city/location name from the user query.

    Args:
        user_question (str): The user's natural language question.

    Returns:
        str | None: Extracted location name or None if not found.
    """
    q = user_question.strip()
    if not q:
        return None

    # Words we don't want to mistake as locations
    WEATHER_WORDS = {
        "weather", "temperature", "temp", "rain", "snow", "precipitation",
        "storm", "umbrella", "forecast", "climate", "conditions", "hot", "cold",
        "warm", "wet", "dry"
    }

    # --- 1. Regex: explicit pattern "in [city]" or "for [city]"
    pattern = re.compile(
        r"\b(?:in|for|at)\s+([A-Za-z\u00C0-\u017F\-\s]+?)(?=\s+(?:" + _STOP_WORDS_PATTERN + r")\b|[?.!,]|$)",
        re.IGNORECASE,
    )
    m = pattern.search(q)
    if m:
        candidate = m.group(1).strip(" ,.?!").lower()
        if candidate not in WEATHER_WORDS:
            return candidate

    # --- 2. Look for capitalized names (e.g., "London", "New York")
    cap_matches = re.findall(r'\b([A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b', q)
    if cap_matches:
        # Exclude any weather/date words
        clean_matches = [
            name for name in cap_matches
            if name.lower() not in WEATHER_WORDS and name.lower() not in DATE_WORDS
        ]
        if clean_matches:
            return max(clean_matches, key=lambda s: len(s.split())).lower()

    # --- 3. Fallback: last non-filler token
    cleaned = re.sub(r'[?.!,;:]', ' ', q)
    tokens = [t for t in cleaned.split() if t.strip()]
    filler = {
        "will", "it", "be", "is", "what", "the", "do", "i", "need",
        "an", "if", "should", "for", "in", "at", "on"
    }
    tokens_filtered = [
        t for t in tokens
        if t.lower() not in filler and t.lower() not in WEATHER_WORDS and t.lower() not in DATE_WORDS
    ]

    if not tokens_filtered:
        return None

    candidate = tokens_filtered[-1].lower()
    return candidate if candidate not in WEATHER_WORDS else None



# --- main function ---
def parse_weather_question(user_question):
    """
    Parse a weather question into structured info and fetch weather data using get_weather_data().

    Returns dict with keys:
      - intent: 'precipitation' | 'temperature' | 'condition'
      - location: string (or None)
      - date: 'YYYY-MM-DD' (string)
      - date_obj: datetime.date object (useful internally)
      - raw_weather: dict returned by get_weather_data (or {} on failure)
      - forecast_match: daily forecast dict that matches the requested date (or None)
    """
    if not isinstance(user_question, str) or not user_question.strip():
        raise ValueError("user_question must be a non-empty string")

    q = user_question.strip()
    q_lower = q.lower()

    # --- 1) intent detection (simple keywords) ---
    if any(w in q_lower for w in ["rain", "snow", "precip", "umbrella", "wet", "drizzle", "storm"]):
        intent = "precipitation"
    elif any(w in q_lower for w in ["temp", "temperature", "hot", "cold", "warm", "degrees", "°c", "°f"]):
        intent = "temperature"
    else:
        intent = "condition"

    # --- 2) date parsing ---
    date_obj = _parse_date_from_text(q)
    date_str = date_obj.isoformat()

    # --- 3) location extraction ---
    location = _extract_location(q)
    # fallback default if none found (you may change this to None or IP-based)
    if not location:
        # safer to require location? For now return None but still attempt to fetch local weather if get_weather_data supports empty => remote IP
        location = None

    # --- 4) determine how many forecast days to request from wttr.in ---
    today = datetime.today().date()
    days_needed = (date_obj - today).days + 1  # e.g., today->1, tomorrow->2
    # clamp to 1..5 (wttr.in provides up to 5 days as per our earlier code)
    if days_needed < 1:
        days_needed = 1
    if days_needed > 5:
        days_needed = 5

    # --- 5) fetch weather (call your existing get_weather_data) ---
    raw_weather = {}
    forecast_match = None
    if location:
        try:
            raw_weather = get_weather_data(location, forecast_days=days_needed)
        except Exception as e:
            # get_weather_data should already handle requests exceptions; keep safe here
            print(f"Error calling get_weather_data: {e}")
            raw_weather = {}

    else:
        # no location found; you can choose to raise, default, or attempt local IP-based lookup.
        print("No location detected in the question. Please include a city (e.g., 'in Tokyo').")
        raw_weather = {}

    # --- 6) try to match requested date to the day's forecast (wttr.in returns 'date' as 'YYYY-MM-DD' strings) ---
    if raw_weather and "forecast" in raw_weather:
        for day in raw_weather["forecast"]:
            if day.get("date") == date_str:
                forecast_match = day
                break
        # if exact date not found, attempt a best-effort fallback (first forecast day)
        if not forecast_match and raw_weather["forecast"]:
            forecast_match = raw_weather["forecast"][0]  # default to first day

    result = {
        "intent": intent,
        "location": location,
        "date": date_str,
        "date_obj": date_obj,
        "raw_weather": raw_weather,
        "forecast_match": forecast_match,
    }
    return result


## Generate Weather Response

In [60]:
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data (parsed into 'location', 'current', 'forecast')

    Returns:
        str: Natural language response
    """
    intent = parsed_question.get("intent")
    location = parsed_question.get("location", "the specified location")
    date = parsed_question.get("date")

    # Safety: check if we even have a forecast
    forecast_days = weather_data.get("forecast", [])
    chosen_day = None
    for day in forecast_days:
        if day.get("date") == str(date):   # match exact date
            chosen_day = day
            break

    if not chosen_day:
        return f"Sorry, I couldn't find a forecast for {date} in {location}."

    # Handle intents
    if intent in ["rain", "precipitation"]:
        # take the last hourly slot (usually night) with rain chance
        hourly = chosen_day.get("hourly", [])
        if hourly:
            best_slot = max(hourly, key=lambda h: int(h.get("chance_of_rain", 0)))
            chance = best_slot.get("chance_of_rain", "0")
            condition = best_slot.get("condition", "").strip()
            return f"In {location.title()} on {date}, there is a {chance}% chance of rain ({condition})."
        return f"In {location.title()} on {date}, I couldn't find rain information."

    elif intent == "temperature":
        max_temp = chosen_day.get("max_temp_C", "N/A")
        min_temp = chosen_day.get("min_temp_C", "N/A")
        return f"The temperature in {location.title()} on {date} will range from {min_temp}°C to {max_temp}°C."

    elif intent == "weather":
        hourly = chosen_day.get("hourly", [])
        if hourly:
            mid_day = hourly[len(hourly)//2]
            condition = mid_day.get("condition", "unknown").strip()
            return f"The weather in {location.title()} on {date} is expected to be {condition}."
        return f"The weather forecast in {location.title()} on {date} is unavailable."

    else:
        # fallback summary
        return (
            f"In {location.title()} on {date}, "
            f"temps will range {chosen_day.get('min_temp_C', 'N/A')}°C–{chosen_day.get('max_temp_C', 'N/A')}°C "
            f"with some chance of rain."
        )


## 📊 Visualisation Functions

In [61]:


# ------------------ Precipitation Visualisation ------------------
def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualization of precipitation chances.

    Args:
        weather_data (dict): The processed weather data with 'forecast'
        output_type (str): Either 'display' (show plot) or 'figure' (return figure object)

    Returns:
        If output_type == 'figure', returns the matplotlib figure object
        Otherwise, displays the visualization in the notebook
    """
    forecast = weather_data.get("forecast", [])
    if not forecast:
        print("No forecast data available for precipitation visualization.")
        return None

    # Take daily average chance of rain
    dates = []
    avg_precip = []
    for f in forecast:
        dates.append(f["date"])
        hourly = f.get("hourly", [])
        if hourly:
            chances = [int(h.get("chance_of_rain", 0)) for h in hourly]
            avg_precip.append(sum(chances) // len(chances))
        else:
            avg_precip.append(0)

    # Plot
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(dates, avg_precip, color="skyblue", alpha=0.7)

    ax.set_title("Average Daily Chance of Rain")
    ax.set_xlabel("Date")
    ax.set_ylabel("Chance of Rain (%)")
    ax.set_ylim(0, 100)
    ax.grid(True, linestyle="--", alpha=0.6, axis="y")

    if output_type == "figure":
        return fig
    else:
        plt.show()
        return None


In [62]:

# ------------------ Temperature Visualisation ------------------
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualization of temperature data.

    Args:
        weather_data (dict): The processed weather data with 'forecast'
        output_type (str): Either 'display' (show plot) or 'figure' (return figure object)

    Returns:
        If output_type == 'figure', returns the matplotlib figure object
        Otherwise, displays the visualization in the notebook
    """
    forecast = weather_data.get("forecast", [])
    if not forecast:
        print("No forecast data available for temperature visualization.")
        return None

    # Extract dates and temperature ranges
    dates = [f["date"] for f in forecast]
    min_temps = [int(f["min_temp_C"]) for f in forecast]
    max_temps = [int(f["max_temp_C"]) for f in forecast]

    # Plot
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.plot(dates, min_temps, marker="o", label="Min Temp (°C)")
    ax.plot(dates, max_temps, marker="o", label="Max Temp (°C)")
    ax.fill_between(dates, min_temps, max_temps, color="lightblue", alpha=0.3)

    ax.set_title("Temperature Forecast")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    ax.grid(True, linestyle="--", alpha=0.6)

    if output_type == "figure":
        return fig
    else:
        plt.show()
        return None




## 🧪 Testing and Examples

## ❗ Test Cases for generate_weather_response

In [ ]:
# question = "What's the precipitation chance in Tokyo?"
# parsed = parse_weather_question(question)
# print(parsed)
# weather = get_weather_data("Tokyo", forecast_days=3)
# print(weather)
# response = generate_weather_response(parsed, weather)
# print(response)


## ❗ Test Cases for pars_weather_data

In [ ]:
# Test 1: Simple Valid Questions
# Basic functionality
# result = parse_weather_question("What's the weather in Tokyo?")
# print(result)  # Should be "Tokyo"
# print(result["intent"])    # Should be "condition"

# Test 2: Date Parsing
# result = parse_weather_question("Will it rain tomorrow in Sydney?")
# print(result["date"])      # Tomorrow's date
# print(result["intent"])    # Should be "precipitation"

# # Test 3: Temperature Questions
# result = parse_weather_question("How hot will it be in London?")
# print(result["intent"])    # Should be "temperature"
# print(result["location"])  # Should be "London"

# # Test 4: Invalid Inputs
# # Empty string
# # result = parse_weather_question("")  # Will raise ValueError

# # None input
# # result = parse_weather_question(None)  # Will raise ValueError
# # Test 5: No Location Questions
# result = parse_weather_question("Will it rain tomorrow?")
# print(result["location"])     # Will be None
# print(result["raw_weather"])  # Will be empty {}

In [ ]:
# # Test 6: Intent Detection Edge Cases
# edge_intents = [
#     ("Will I need sunscreen?", "condition"),        # Not obvious category
#     ("Is it freezing cold?", "temperature"),        # Should detect "cold"
#     ("Chance of precipitation?", "precipitation"),   # Should detect "precip"
#     ("Weather conditions?", "condition"),           # Generic
#     ("Hot or cold tomorrow?", "temperature"),       # Multiple keywords
# ]

# for question, expected in edge_intents:
#     result = parse_weather_question(question)
#     print(f"'{question}' -> {result['intent']} (expected: {expected})")
# # Test 7: Location Extraction Failures
# location_tests = [
#     ("weather in tokyo", None),           # Lowercase - current regex misses
#     ("Tokyo weather forecast", "Tokyo"),   # No "in" keyword
#     ("weather for new york", None),       # Uses "for" not "in"
#     ("london temperature", "london"),     # No preposition
#     ("weather in new york, usa", "new york"),  # With country code
# ]

# for question, expected in location_tests:
#     result = parse_weather_question(question)
#     actual = result["location"]
#     print(f"'{question}' -> '{actual}' (expected: '{expected}')")
#     if actual != expected:
#         print("  FAILED!")
# # Test 8: Date Calculation Problems
# from datetime import date, timedelta

# # Test date boundaries
# today = date.today()
# test_dates = [
#     ("weather today", today),
#     ("weather tomorrow", today + timedelta(days=1)),
#     ("weather on monday", None),  # Depends on current day
# ]

# for question, expected in test_dates:
#     result = parse_weather_question(question)
#     print(f"'{question}' -> {result['date']} (expected: {expected})")
# # Test 9: Missing Dependencies
# # If get_weather_data is not defined:
# def test_missing_dependency():
#     # This will crash with NameError
#     result = parse_weather_question("weather in Tokyo")
#     # NameError: name 'get_weather_data' is not defined
# # Test 10: Complex Real Questions
# complex_questions = [
#     "Should I bring an umbrella to work in sydney tomorrow morning?",
#     "What's the temperature going to be like in new york this weekend?",
#     "Is it going to be hot in los angeles next tuesday?",
#     "Rain forecast for melbourne, australia tomorrow?",
#     "How cold will it be in moscow on friday?",
# ]

# for q in complex_questions:
#     try:
#         result = parse_weather_question(q)
#         print(f"Question: {q}")
#         print(f"Location: {result['location']}")
#         print(f"Intent: {result['intent']}")
#         print(f"Date: {result['date']}")
#         print("---")
#     except Exception as e:
#         print(f"CRASHED on: {q}")
#         print(f"Error: {e}")

## ❗ Test Cases for get_weather_data




In [ ]:
# # Test 1: Normal Usage
# data = get_weather_data("Tokyo", forecast_days=3)
# print(data["current"])
# print(data["forecast"][0])

# # Test 2: Invalid City
# data = get_weather_data("FakeCity123", forecast_days=3)
# print(data)  # Will print {} - but is this an error or empty data?

# # Test 3: Empty Location
# data = get_weather_data("", forecast_days=3)
# print(data)

# # Test 4: Bad forecast_days
# data = get_weather_data("Sydney", forecast_days=10)  # Should raise ValueError

# # Test 5: Special Characters
# data = get_weather_data("São Paulo", forecast_days=2)
# print(data["current"]["temperature_C"])

## ❗ Test Cases for visualization



In [ ]:
# Step 1: Get real weather data
# weather_data = get_weather_data("Tokyo", forecast_days=3)

# # Step 2: Plot temperature and precipitation
# create_temperature_visualisation(weather_data)
# create_precipitation_visualisation(weather_data)
